##### The cell below is for you to keep track of the libraries used and install those libraries quickly
##### Ensure that the proper library names are used and the syntax of `%pip install PACKAGE_NAME` is followed

In [ ]:
# %pip install pandas 
# %pip install matplotlib
# %pip install graphviz
# %pip install h2o
# add commented pip installation lines for packages used as shown above for ease of testing
# the line should be of the format %pip install PACKAGE_NAME 

## **DO NOT CHANGE** the filepath variable
##### Instead, create a folder named 'data' in your current working directory and 
##### have the .parquet file inside that. A relative path *must* be used when loading data into pandas

In [2]:
# Can have as many cells as you want for code
filepath = "./data/catB_train.parquet" 
# the initialised filepath MUST be a relative path to a folder named data that contains the parquet file

### **ALL** Code for machine learning and dataset analysis should be entered below. 
##### Ensure that your code is clear and readable.
##### Comments and Markdown notes are advised to direct attention to pieces of code you deem useful.

## The cell below is **NOT** to be removed
##### The function is to be amended so that it accepts the given input (dataframe) and returns the required output (list). 
##### It is recommended to test the function out prior to submission
-------------------------------------------------------------------------------------------------------------------------------
##### The hidden_data parsed into the function below will have the same layout columns wise as the dataset *SENT* to you
##### Thus, ensure that steps taken to modify the initial dataset to fit into the model are also carried out in the function below

In [3]:
## IMPORTING LIBRARIES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h2o
from h2o.estimators import H2ORandomForestEstimator

In [ ]:
#Read data
df = pd.read_parquet(filepath)

#Reading the data and having an overview
df.head()

In [ ]:
def testing_hidden_data(hidden_data: pd.DataFrame) -> list:
    '''DO NOT REMOVE THIS FUNCTION.

The function accepts a dataframe as input and return an iterable (list)
of binary classes as output.

The function should be coded to test on hidden data
and should include any preprocessing functions needed for your model to perform. 
    
All relevant code MUST be included in this function.'''
    df = hidden_data

    ## Cleaning

    #1. Search for duplicates
  
    #2. Drop "clntnum" column
    df.drop("clntnum", axis=1, inplace=True)
    
    #3. Drop columns that have only one value
    for col in df:
        if df[col].nunique(dropna=False) == 1:
            df.drop(col, axis=1, inplace=True)
             #57 columns dropped

    df.isnull().sum().to_string()

    #4. Drop the 1014 rows with NaN for the 20 key columns (not very informative samples)
    df = df[~df["flg_substandard"].isna()]
    
    #Studying the columns with null values
    for column in df.columns:
        if df[column].isnull().any():
            p = df[column].isnull().sum()/16978*100
            unique_values = df[column].unique()
                        
    #imputation by taking a random sample from non-null values in the column
    def impute(df, column_to_impute):
        non_null_values = df[column_to_impute].dropna().values
        null_indices = df[df[column_to_impute].isnull()].index
        df.loc[null_indices, column_to_impute] = np.random.choice(non_null_values, size=len(null_indices), replace=True)

    #race_desc can use imputation by random sampling from non-null values.
    impute(df, "race_desc")

    #ctrycode_desc can change to "Not Applicable".
    df['ctrycode_desc'].fillna("Not Applicable", inplace=True)

    #cltsex_fix can use imputation.
    impute(df, "cltsex_fix")

    #f_ever_declined_la can change to False (1 change to True)
    df['f_ever_declined_la'].fillna(False, inplace=True)
    df['f_ever_declined_la'] = df['f_ever_declined_la'].astype(bool)

    #hh_size can be dropped (duplicate of hh_size_est)
    a = df.shape[1]
    df.drop("hh_size", axis=1, inplace=True)

    #hh_size_est can use imputation.
    impute(df, "hh_size_est")

    #annual_income_est can use imputation.
    impute(df, "annual_income_est")

    a = df.shape[1]
    df.drop("hh_20", axis=1, inplace=True)

    a = df.shape[1]
    df.drop("pop_20", axis=1, inplace=True)
 

    #recency_lapse can be converted to cat var, replace nan with "no lapse".
        #can use bins of range 50 up to >400. 
        #Here we notice that for all the columns that have 11773 nan values, they refer to the 11773 people who do not have lapsed policies.
    bins = [0, 50, 100, 150, 200, 250, 300, 350, 400, float('inf')]
    labels = ['0-50', '51-100', '101-150', '151-200', '201-250', '251-300', '301-350', '351-400', '>400']
    df['recency_lapse'] = pd.cut(df['recency_lapse'], bins=bins, labels=labels, right=False)
    df['recency_lapse'] = df['recency_lapse'].cat.add_categories("No lapse").fillna("No lapse")
 
        
    #recency_cancel can be converted to cat var, replace nan with "no cancel".
        #can use bins of range 20 up to >100. 
        #Similarly, we notice there are 16376 people who did not cancel any policy.
    bins = [0, 20, 40, 60, 80, 100, float('inf')]
    labels = ['0-20', '21-40', '41-60', '61-80', '81-100', '>100']
    df['recency_cancel'] = pd.cut(df['recency_cancel'], bins=bins, labels=labels, right=False)
    df['recency_cancel'] = df['recency_cancel'].cat.add_categories("No cancel").fillna("No cancel")

    #tot_cancel_pols can change to 0 (stands for no policy cancelled).
    df['tot_cancel_pols'].fillna(0, inplace=True)

    #drop the lapse_ape columns that have less than 10 different values (nonzero, nonnull)
    #for the others, convert to cat var (category for 0s; replace None with "No lapse")

    #drop the n_months_since_lapse columns that have less than 10 different values (non-ve, non-9999, nonnull)
    #for the others, convert to cat var (replace -ve, 9999 with "No data"; replace None with "No lapse")

    #dropping the lapse_ape and n_months_since_lapse from above
    cols = ["lapse_ape_ltc_1280bf","lapse_ape_grp_de05ae","lapse_ape_inv_dcd836","lapse_ape_lh_d0adeb","lapse_ape_inv_e9f316",
        "lapse_ape_32c74c","n_months_since_lapse_ltc_1280bf","n_months_since_lapse_grp_de05ae","n_months_since_lapse_inv_dcd836",
        "n_months_since_lapse_lh_d0adeb","n_months_since_lapse_inv_e9f316","n_months_since_lapse_32c74c"]
    a = df.shape[1]
    df.drop(cols, axis=1, inplace=True)


    #lapse_ape_grp_6fc3e6
    bins = [0, 0.01, 100, 200, float('inf')]
    labels = ['Zero', '>0-100', '101-200', '>200']
    df['lapse_ape_grp_6fc3e6'].fillna(-1, inplace=True)
    df['lapse_ape_grp_6fc3e6'] = pd.cut(df['lapse_ape_grp_6fc3e6'], bins=bins, labels=labels, right=False)
    df['lapse_ape_grp_6fc3e6'] = df['lapse_ape_grp_6fc3e6'].cat.add_categories("No lapse").fillna("No lapse")

    #lapse_ape_grp_945b5a
    bins = [0, 0.01, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, float('inf')]
    labels = ['Zero', '>0-100', '101-200', '201-300', '301-400', '401-500', '501-600', '601-700', '701-800', '801-900', '901-1000', '>1000']
    df['lapse_ape_grp_945b5a'].fillna(-1, inplace=True)
    df['lapse_ape_grp_945b5a'] = pd.cut(df['lapse_ape_grp_945b5a'], bins=bins, labels=labels, right=False)
    df['lapse_ape_grp_945b5a'] = df['lapse_ape_grp_945b5a'].cat.add_categories("No lapse").fillna("No lapse")
  
    #lapse_ape_grp_6a5788 
    bins = [0, 0.01, 200, 400, 600, float('inf')]
    labels = ['Zero', '>0-200', '201-400', '401-600', '>600']
    df['lapse_ape_grp_6a5788'].fillna(-1, inplace=True)
    df['lapse_ape_grp_6a5788'] = pd.cut(df['lapse_ape_grp_6a5788'], bins=bins, labels=labels, right=False)
    df['lapse_ape_grp_6a5788'] = df['lapse_ape_grp_6a5788'].cat.add_categories("No lapse").fillna("No lapse")

    #lapse_ape_ltc_43b9d5 
    bins = [0, 0.01, 200, 400, 600, 800, 1000, float('inf')]
    labels = ['Zero', '>0-200', '201-400', '401-600', '601-800', '801-1000', '>1000']
    df['lapse_ape_ltc_43b9d5'].fillna(-1, inplace=True)
    df['lapse_ape_ltc_43b9d5'] = pd.cut(df['lapse_ape_ltc_43b9d5'], bins=bins, labels=labels, right=False)
    df['lapse_ape_ltc_43b9d5'] = df['lapse_ape_ltc_43b9d5'].cat.add_categories("No lapse").fillna("No lapse")
    
    #lapse_ape_grp_9cdedf 
    bins = [0, 0.01, 100, 200, 300, 400, 500, float('inf')]
    labels = ['Zero', '>0-100', '101-200', '201-300', '301-400', '401-500', '>500']
    df['lapse_ape_grp_9cdedf'].fillna(-1, inplace=True)
    df['lapse_ape_grp_9cdedf'] = pd.cut(df['lapse_ape_grp_9cdedf'], bins=bins, labels=labels, right=False)
    df['lapse_ape_grp_9cdedf'] = df['lapse_ape_grp_9cdedf'].cat.add_categories("No lapse").fillna("No lapse")

    #lapse_ape_grp_1581d7
    bins = [0, 0.01, 50, 100, 150, 200, 250, float('inf')]
    labels = ['Zero', '>0-50', '51-100', '101-150', '151-200', '200-250', '>250']
    df['lapse_ape_grp_1581d7'].fillna(-1, inplace=True)
    df['lapse_ape_grp_1581d7'] = pd.cut(df['lapse_ape_grp_1581d7'], bins=bins, labels=labels, right=False)
    df['lapse_ape_grp_1581d7'] = df['lapse_ape_grp_1581d7'].cat.add_categories("No lapse").fillna("No lapse")
   
    #lapse_ape_grp_22decf
    bins = [0, 0.01, 500, 1000, 1500, 2000, float('inf')]
    labels = ['Zero', '>0-500', '501-1000', '1001-1500', '1501-2000','>2000']
    df['lapse_ape_grp_22decf'].fillna(-1, inplace=True)
    df['lapse_ape_grp_22decf'] = pd.cut(df['lapse_ape_grp_22decf'], bins=bins, labels=labels, right=False)
    df['lapse_ape_grp_22decf'] = df['lapse_ape_grp_22decf'].cat.add_categories("No lapse").fillna("No lapse")

    #lapse_ape_lh_507c37
    bins = [0, 0.01, 500, 1000, 1500, 2000, 3000, 4000, 5000, float('inf')]
    labels = ['Zero', '>0-500', '501-1000', '1001-1500', '1501-2000','2001-3000','3001-4000','4001-5000','>5000']
    df['lapse_ape_lh_507c37'].fillna(-1, inplace=True)
    df['lapse_ape_lh_507c37'] = pd.cut(df['lapse_ape_lh_507c37'], bins=bins, labels=labels, right=False)
    df['lapse_ape_lh_507c37'] = df['lapse_ape_lh_507c37'].cat.add_categories("No lapse").fillna("No lapse")
 
    #lapse_ape_lh_839f8a
    bins = [0, 0.01, 1000, 1500, 2000, 2500, 3000, 4000, 5000, float('inf')]
    labels = ['Zero', '>0-1000', '1001-1500', '1501-2000', '2001-2500', '2501-3000', '3001-4000', '4001-5000', '>5000']
    df['lapse_ape_lh_839f8a'].fillna(-1, inplace=True)
    df['lapse_ape_lh_839f8a'] = pd.cut(df['lapse_ape_lh_839f8a'], bins=bins, labels=labels, right=False)
    df['lapse_ape_lh_839f8a'] = df['lapse_ape_lh_839f8a'].cat.add_categories("No lapse").fillna("No lapse")
        
    #lapse_ape_grp_caa6ff
    bins = [0, 0.01, 100, 200, 300, 400, 500, 600, 700, float('inf')]
    labels = ['Zero', '>0-100', '101-200', '201-300', '301-400', '401-500', '501-600', '601-700', '>700']
    df['lapse_ape_grp_caa6ff'].fillna(-1, inplace=True)
    df['lapse_ape_grp_caa6ff'] = pd.cut(df['lapse_ape_grp_caa6ff'], bins=bins, labels=labels, right=False)
    df['lapse_ape_grp_caa6ff'] = df['lapse_ape_grp_caa6ff'].cat.add_categories("No lapse").fillna("No lapse")

    #lapse_ape_grp_fd3bfb
    bins = [0, 0.01, 200, 400, 600, float('inf')]
    labels = ['Zero', '>0-200', '201-400', '401-600', '>600']
    df['lapse_ape_grp_fd3bfb'].fillna(-1, inplace=True)
    df['lapse_ape_grp_fd3bfb'] = pd.cut(df['lapse_ape_grp_fd3bfb'], bins=bins, labels=labels, right=False)
    df['lapse_ape_grp_fd3bfb'] = df['lapse_ape_grp_fd3bfb'].cat.add_categories("No lapse").fillna("No lapse")
 
    #lapse_ape_lh_e22a6a
    bins = [0, 0.01, 400, 800, 1200, 1600, 2000, 3000, 4000, float('inf')]
    labels = ['Zero', '>0-400', '401-800', '801-1200', '1201-1600', '1601-2000', '2001-3000', '3001-4000', '>4000']
    df['lapse_ape_lh_e22a6a'].fillna(-1, inplace=True)
    df['lapse_ape_lh_e22a6a'] = pd.cut(df['lapse_ape_lh_e22a6a'], bins=bins, labels=labels, right=False)
    df['lapse_ape_lh_e22a6a'] = df['lapse_ape_lh_e22a6a'].cat.add_categories("No lapse").fillna("No lapse")
  
    #lapse_ape_grp_70e1dd
    bins = [0, 0.01, 100, 200, 300, 400, 500, 600, float('inf')]
    labels = ['Zero', '>0-100', '101-200', '201-300', '301-400', '401-500', '501-600', '>600']
    df['lapse_ape_grp_70e1dd'].fillna(-1, inplace=True)
    df['lapse_ape_grp_70e1dd'] = pd.cut(df['lapse_ape_grp_70e1dd'], bins=bins, labels=labels, right=False)
    df['lapse_ape_grp_70e1dd'] = df['lapse_ape_grp_70e1dd'].cat.add_categories("No lapse").fillna("No lapse")
        
    #lapse_ape_grp_e04c3a
    bins = [0, 0.01, 500, 1000, 1500, 2000, float('inf')]
    labels = ['Zero', '>0-500', '501-1000', '1001-1500', '1501-2000', '>2000']
    df['lapse_ape_grp_e04c3a'].fillna(-1, inplace=True)
    df['lapse_ape_grp_e04c3a'] = pd.cut(df['lapse_ape_grp_e04c3a'], bins=bins, labels=labels, right=False)
    df['lapse_ape_grp_e04c3a'] = df['lapse_ape_grp_e04c3a'].cat.add_categories("No lapse").fillna("No lapse")

        
    #lapse_ape_grp_fe5fb8
    bins = [0, 0.01, 200, 400, 600, 800, 1000, 1200, 1400, 1600, float('inf')]
    labels = ['Zero', '>0-200', '201-400', '401-600', '601-800', '801-1000', '1001-1200', '1201-1400', '1401-1600', '>1600']
    df['lapse_ape_grp_fe5fb8'].fillna(-1, inplace=True)
    df['lapse_ape_grp_fe5fb8'] = pd.cut(df['lapse_ape_grp_fe5fb8'], bins=bins, labels=labels, right=False)
    df['lapse_ape_grp_fe5fb8'] = df['lapse_ape_grp_fe5fb8'].cat.add_categories("No lapse").fillna("No lapse")

    #lapse_ape_grp_94baec
    bins = [0, 0.01, 100, 200, 400, 600, 800, 1000, float('inf')]
    labels = ['Zero', '>0-100', '101-200', '201-400', '401-600', '601-800', '801-1000', '>1000']
    df['lapse_ape_grp_94baec'].fillna(-1, inplace=True)
    df['lapse_ape_grp_94baec'] = pd.cut(df['lapse_ape_grp_94baec'], bins=bins, labels=labels, right=False)
    df['lapse_ape_grp_94baec'] = df['lapse_ape_grp_94baec'].cat.add_categories("No lapse").fillna("No lapse")
        
    #lapse_ape_grp_e91421
    bins = [0, 0.01, 200, 400, 600, 800, 1000, float('inf')]
    labels = ['Zero', '>0-200', '201-400', '401-600', '601-800', '801-1000', '>1000']
    df['lapse_ape_grp_e91421'].fillna(-1, inplace=True)
    df['lapse_ape_grp_e91421'] = pd.cut(df['lapse_ape_grp_e91421'], bins=bins, labels=labels, right=False)
    df['lapse_ape_grp_e91421'] = df['lapse_ape_grp_e91421'].cat.add_categories("No lapse").fillna("No lapse")

        
    #lapse_ape_lh_f852af
    bins = [0, 0.01, 500, 1000, 1500, 2000, 2500, 3000, 4000, 6000, 8000, float('inf')]
    labels = ['Zero', '>0-500', '501-1000', '1001-1500', '1501-2000', '2001-2500', '2501-3000', '3001-4000', '4001-6000', '6001-8000', '>8000']
    df['lapse_ape_lh_f852af'].fillna(-1, inplace=True)
    df['lapse_ape_lh_f852af'] = pd.cut(df['lapse_ape_lh_f852af'], bins=bins, labels=labels, right=False)
    df['lapse_ape_lh_f852af'] = df['lapse_ape_lh_f852af'].cat.add_categories("No lapse").fillna("No lapse")

        
    #lapse_ape_lh_947b15
    bins = [0, 0.01, 1000, 2000, 3000, 4000, 5000, 6000, float('inf')]
    labels = ['Zero', '>0-1000', '1001-2000', '2001-3000', '3001-4000', '4001-5000', '5001-6000', '>6000']
    df['lapse_ape_lh_947b15'].fillna(-1, inplace=True)
    df['lapse_ape_lh_947b15'] = pd.cut(df['lapse_ape_lh_947b15'], bins=bins, labels=labels, right=False)
    df['lapse_ape_lh_947b15'] = df['lapse_ape_lh_947b15'].cat.add_categories("No lapse").fillna("No lapse")

        
    #converting n_months_since_lapse columns to cat var
    cols=['n_months_since_lapse_grp_6fc3e6', 'n_months_since_lapse_grp_945b5a', 'n_months_since_lapse_grp_6a5788', 
    'n_months_since_lapse_ltc_43b9d5', 'n_months_since_lapse_grp_9cdedf',
    'n_months_since_lapse_grp_1581d7', 'n_months_since_lapse_grp_22decf', 'n_months_since_lapse_lh_507c37', 
    'n_months_since_lapse_lh_839f8a', 'n_months_since_lapse_grp_caa6ff', 
    'n_months_since_lapse_grp_fd3bfb', 'n_months_since_lapse_lh_e22a6a', 'n_months_since_lapse_grp_70e1dd', 
    'n_months_since_lapse_grp_e04c3a', 'n_months_since_lapse_grp_fe5fb8', 'n_months_since_lapse_grp_94baec', 
    'n_months_since_lapse_grp_e91421', 'n_months_since_lapse_lh_f852af', 'n_months_since_lapse_lh_947b15']

    bins = [0, 20, 40, 60, 80, 100, float('inf')]
    labels = ['0-20', '21-40', '41-60', '61-80', '81-100', '>100']

    for col in cols:
        df[col] = df[col].astype('float64').replace(9999,-1)
        null_indices = df[col].index[df[col].isnull()].tolist()
        df[col] = pd.cut(df[col], bins=bins, labels=labels, right=False)
        df[col] = df[col].cat.add_categories("No lapse")
        df[col].loc[null_indices] = "No lapse"
        df[col] = df[col].cat.add_categories("No data").fillna("No data")


    #checking the distribution of columns, for determining the bins when converting to categorical variable

    #flg_affconnect_show_interest_ever can change to False (1 change to True)
    df['flg_affconnect_show_interest_ever'].fillna(False, inplace=True)
    df['flg_affconnect_show_interest_ever'] = df['flg_affconnect_show_interest_ever'].astype(bool)


    #flg_affconnect_ready_to_buy_ever can be dropped (very similar to n_months_since_visit_affcon)
    a = df.shape[1]
    df.drop("flg_affconnect_ready_to_buy_ever", axis=1, inplace=True)
   

    #flg_affconnect_lapse_ever can be dropped (duplicate of have_lapse and flg_affconnect_ready_to_buy_ever)
    a = df.shape[1]
    df.drop("flg_affconnect_lapse_ever", axis=1, inplace=True)


    #affcon_visit_days can be changed to 0
        #Here we can hypothesise that 16176 people did not buy through affcon
    df['affcon_visit_days'].fillna(0, inplace=True)
 
        
    #n_months_since_visit_affcon can be converted to cat var, replace nan with "no affcon".
    df['n_months_since_visit_affcon'] = df['n_months_since_visit_affcon'].astype('category')
    df['n_months_since_visit_affcon'] = df['n_months_since_visit_affcon'].cat.add_categories("No affcon").fillna("No affcon")


    #clmcon_visit_days can be changed to 0
        #Here we can hypothesise that 16279 people did not use clmcon
    df['clmcon_visit_days'].fillna(0, inplace=True)
 
        
    #recency_clmcon, recency_clmcon_regis can be converted to cat var, replace nan with "no clmcon"
        #recency_clmcon_regis can use bins of range 20
    df['recency_clmcon'] = df['recency_clmcon'].astype('category')
    df['recency_clmcon'] = df['recency_clmcon'].cat.add_categories("No clmcon").fillna("No clmcon")
  

    bins = [0, 10, 20, 30, 40, 60, 80, float('inf')]
    labels = ['0-10', '11-20', '21-30', '31-40', '41-60', '61-80', '>80']
    df['recency_clmcon_regis'] = pd.cut(df['recency_clmcon_regis'], bins=bins, labels=labels, right=False)
    df['recency_clmcon_regis'] = df['recency_clmcon_regis'].cat.add_categories("No clmcon").fillna("No clmcon")
    
        
    #hlthclaim_amt can be converted to cat var, replace nan with "no hlthclaim"
        #Low: $0 - $500; Medium Low: $501 - $1,000; Medium: $1,001 - $5,000; High: $5,001 - $10,000; Very High: $10,001 and above
        #Here we can hypothesise that 15552 people did not make healthclaims
    bins = [0, 500, 1000, 5000, 10000, float('inf')]
    labels = ['Low: $0 - $500', 'Medium Low: $501 - $1,000', 'Medium: $1,001 - $5,000', 'High: $5,001 - $10,000', 'Very High: >$10,000']
    df['hlthclaim_amt'] = df['hlthclaim_amt'].fillna(-1)
    df['hlthclaim_amt'] = pd.cut(df['hlthclaim_amt'], bins=bins, labels=labels, right=False)
    df['hlthclaim_amt'] = df['hlthclaim_amt'].cat.add_categories("No healthclaim").fillna("No healthclaim")
   
        
    #recency_hlthclaim can be converted to cat var, replace nan with "no hlthclaim"
        #can use bins of range 20
    bins = [0, 10, 20, 30, 40, 60, 80, 100, float('inf')]
    labels = ['0-10', '11-20', '21-30', '31-40', '41-60', '61-80', '81-100', '>100']
    df['recency_hlthclaim'] = pd.cut(df['recency_hlthclaim'], bins=bins, labels=labels, right=False)
    df['recency_hlthclaim'] = df['recency_hlthclaim'].cat.add_categories("No healthclaim").fillna("No healthclaim")
 
    #hlthclaim_cnt_success can be converted to cat var, replace nan with "no success"
        #can use bins of ranges 0-20; 21-40; 41-100; 101-180; >180
    bins = [0, 20, 40, 100, 180, float('inf')]
    labels = ['0-20', '21-40', '41-100', '101-180', '>180']
    df['hlthclaim_cnt_success'] = pd.cut(df['hlthclaim_cnt_success'], bins=bins, labels=labels, right=False)
    df['hlthclaim_cnt_success'] = df['hlthclaim_cnt_success'].cat.add_categories("No success").fillna("No success")
  
    #recency_hlthclaim_success can be converted to cat var, replace nan with "no success"
        #can use bins of range 20
    bins = [0, 10, 20, 30, 40, 60, 80, 100, float('inf')]
    labels = ['0-10', '11-20', '21-30', '31-40', '41-60', '61-80', '81-100', '>100']
    df['recency_hlthclaim_success'] = pd.cut(df['recency_hlthclaim_success'], bins=bins, labels=labels, right=False)
    df['recency_hlthclaim_success'] = df['recency_hlthclaim_success'].cat.add_categories("No success").fillna("No success")
    
        
    #hlthclaim_cnt_unsuccess can be converted to cat var, replace nan with "no unsuccess"
    bins = [0, 1, 2, 4, 6, 8, 10, 15, float('inf')]
    labels = ['1', '2', '3-4', '5-6', '7-8', '9-10', '11-15', '>15']
    df['hlthclaim_cnt_unsuccess'] = pd.cut(df['hlthclaim_cnt_unsuccess'], bins=bins, labels=labels, right=False)
    df['hlthclaim_cnt_unsuccess'] = df['hlthclaim_cnt_unsuccess'].cat.add_categories("No unsuccess").fillna("No unsuccess")
 
        
    #recency_hlthclaim_unsuccess can be converted to cat var, replace nan with "no unsuccess"
        #can use bins of range 20
    bins = [0, 20, 40, 60, 80, 100, 120, float('inf')]
    labels = ['0-20', '21-40', '41-60', '61-80', '81-100', '101-120', '>120']
    df['recency_hlthclaim_unsuccess'] = pd.cut(df['recency_hlthclaim_unsuccess'], bins=bins, labels=labels, right=False)
    df['recency_hlthclaim_unsuccess'] = df['recency_hlthclaim_unsuccess'].cat.add_categories("No unsuccess").fillna("No unsuccess")
  

    #flg_hlthclaim_839f8a_ever can change to False (1 change to True)
    df['flg_hlthclaim_839f8a_ever'].fillna(False, inplace=True)
    df['flg_hlthclaim_839f8a_ever'] = df['flg_hlthclaim_839f8a_ever'].astype(bool)


    #recency_hlthclaim_839f8a can be converted to cat var, replace nan with "no claim"
    bins = [0, 2, 5, 10, 20, 40, 60, 80, 100, 120, float('inf')]
    labels = ['0-2', '3-5', '6-10', '11-20', '21-40', '41-60', '61-80', '81-100', '101-120', '>120']
    df['recency_hlthclaim_839f8a'] = pd.cut(df['recency_hlthclaim_839f8a'], bins=bins, labels=labels, right=False)
    df['recency_hlthclaim_839f8a'] = df['recency_hlthclaim_839f8a'].cat.add_categories("No claim").fillna("No claim")

        
    #flg_hlthclaim_14cb37_ever can change to False (1 change to True)
    df['flg_hlthclaim_14cb37_ever'].fillna(False, inplace=True)
    df['flg_hlthclaim_14cb37_ever'] = df['flg_hlthclaim_14cb37_ever'].astype(bool)


    #recency_hlthclaim_14cb37 can be converted to cat var, replace nan with "no claim"
    bins = [0, 2, 5, 10, 20, 40, 60, 80, 100, 120, float('inf')]
    labels = ['0-2', '3-5', '6-10', '11-20', '21-40', '41-60', '61-80', '81-100', '101-120', '>120']
    df['recency_hlthclaim_14cb37'] = pd.cut(df['recency_hlthclaim_14cb37'], bins=bins, labels=labels, right=False)
    df['recency_hlthclaim_14cb37'] = df['recency_hlthclaim_14cb37'].cat.add_categories("No claim").fillna("No claim")

        
    #giclaim_amt can be converted to cat var, replace nan with "no giclaim"
        #Low: $0 - $500; Medium Low: $501 - $1,000; Medium: $1,001 - $5,000; High: $5,001 - $10,000; Very High: $10,001 and above
        #Here we can hypothesise that 16545 people did not make giclaims
    bins = [0, 500, 1000, 5000, 10000, float('inf')]
    labels = ['Low: $0 - $500', 'Medium Low: $501 - $1,000', 'Medium: $1,001 - $5,000', 'High: $5,001 - $10,000', 'Very High: >$10,000']
    df['giclaim_amt'] = df['giclaim_amt'].fillna(-1)
    df['giclaim_amt'] = pd.cut(df['giclaim_amt'], bins=bins, labels=labels, right=False)
    df['giclaim_amt'] = df['giclaim_amt'].cat.add_categories("No giclaim").fillna("No giclaim")
   
        
    #recency_giclaim can be converted to cat var, replace nan with "no giclaim"
        #can use bins of range 10
    bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, float('inf')]
    labels = ['0-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100', '>100']
    df['recency_giclaim'] = pd.cut(df['recency_giclaim'], bins=bins, labels=labels, right=False)
    df['recency_giclaim'] = df['recency_giclaim'].cat.add_categories("No giclaim").fillna("No giclaim")
    
   # Function to convert object columns to categorical
    def convert_to_categorical(df):
        for col in df.columns:
            if df[col].dtype == 'object':
                df[col] = df[col].astype('category')
        return df
    
    ## Export model
    RF_Model = h2o.load_model('./rfTrainedModel')

    df = convert_to_categorical(df)

    result = RF_Model.predict(h2o.H2OFrame(df))
    return result['predict']

##### Cell to check testing_hidden_data function

In [ ]:
# This cell should output a list of predictions.
test_df = pd.read_parquet(filepath)
test_df = test_df.drop(columns=["f_purchase_lh"])
print(testing_hidden_data(test_df))

## The below code block is for training purposes and analysis ONLY. It does not need to be run for obtaining the prediction results.

### Data Cleaning (dealing with nan values)

1. Check for and eliminate duplicate rows.
2. Drop id column.
3. Drop columns with only one value.
4. Drop uninformative samples (missing the 20 key features).
5. Fix null values column by column.

In [ ]:
#1. Search for duplicates
a = df["clntnum"].nunique()
if a == 17992:
    print("There are no duplicates.")
else:
    b = 17992 - a
    print(f"There are {b} duplicates.")



#2. Drop "clntnum" column
df.drop("clntnum", axis=1, inplace=True)
print(df.shape)



#3. Drop columns that have only one value
for col in df:
    if df[col].nunique(dropna=False) == 1:
        df.drop(col, axis=1, inplace=True)
print(df.shape)         #57 columns dropped

In [ ]:
df.isnull().sum().to_string()

In [ ]:
#4. Drop the 1014 rows with NaN for the 20 key columns (not very informative samples)
df = df[~df["flg_substandard"].isna()]
print(df.shape)

#Studying the columns with null values
for column in df.columns:
    if df[column].isnull().any():
        p = df[column].isnull().sum()/16978*100
        unique_values = df[column].unique()
        print(f"{column} has {p}% null values. Unique values: {unique_values}")
        
#imputation by taking a random sample from non-null values in the column
def impute(df, column_to_impute):
    non_null_values = df[column_to_impute].dropna().values
    null_indices = df[df[column_to_impute].isnull()].index
    df.loc[null_indices, column_to_impute] = np.random.choice(non_null_values, size=len(null_indices), replace=True)

In [ ]:
#race_desc can use imputation by random sampling from non-null values.
impute(df, "race_desc")
if df["race_desc"].isnull().sum() == 0:
    print("race_desc done")

#ctrycode_desc can change to "Not Applicable".
df['ctrycode_desc'].fillna("Not Applicable", inplace=True)
if df["ctrycode_desc"].isnull().sum() == 0:
    print("ctrycode_desc done")

#cltsex_fix can use imputation.
impute(df, "cltsex_fix")
if df["cltsex_fix"].isnull().sum() == 0:
    print("cltsex_fix done")

#f_ever_declined_la can change to False (1 change to True)
df['f_ever_declined_la'].fillna(False, inplace=True)
df['f_ever_declined_la'] = df['f_ever_declined_la'].astype(bool)
if df["f_ever_declined_la"].isnull().sum() == 0:
    print("f_ever_declined_la done")

#hh_size can be dropped (duplicate of hh_size_est)
a = df.shape[1]
df.drop("hh_size", axis=1, inplace=True)
if df.shape[1] + 1 == a:
    print("Dropped hh_size")
print(df.shape)

#hh_size_est can use imputation.
impute(df, "hh_size_est")
if df["hh_size_est"].isnull().sum() == 0:
    print("hh_size_est done")

#annual_income_est can use imputation.
impute(df, "annual_income_est")
if df["annual_income_est"].isnull().sum() == 0:
    print("annual_income_est done")
    
#dk what the col mean: hh_20, pop_20 (just drop)
a = df.shape[1]
df.drop("hh_20", axis=1, inplace=True)
if df.shape[1] + 1 == a:
    print("Dropped hh_20")
print(df.shape)

a = df.shape[1]
df.drop("pop_20", axis=1, inplace=True)
if df.shape[1] + 1 == a:
    print("Dropped pop_20")
print(df.shape)

#recency_lapse can be converted to cat var, replace nan with "no lapse".
    #can use bins of range 50 up to >400. 
    #Here we notice that for all the columns that have 11773 nan values, they refer to the 11773 people who do not have lapsed policies.
bins = [0, 50, 100, 150, 200, 250, 300, 350, 400, float('inf')]
labels = ['0-50', '51-100', '101-150', '151-200', '201-250', '251-300', '301-350', '351-400', '>400']
df['recency_lapse'] = pd.cut(df['recency_lapse'], bins=bins, labels=labels, right=False)
df['recency_lapse'] = df['recency_lapse'].cat.add_categories("No lapse").fillna("No lapse")
if df["recency_lapse"].isnull().sum() == 0:
    print("recency_lapse done")
    
#recency_cancel can be converted to cat var, replace nan with "no cancel".
    #can use bins of range 20 up to >100. 
    #Similarly, we notice there are 16376 people who did not cancel any policy.
bins = [0, 20, 40, 60, 80, 100, float('inf')]
labels = ['0-20', '21-40', '41-60', '61-80', '81-100', '>100']
df['recency_cancel'] = pd.cut(df['recency_cancel'], bins=bins, labels=labels, right=False)
df['recency_cancel'] = df['recency_cancel'].cat.add_categories("No cancel").fillna("No cancel")
if df["recency_cancel"].isnull().sum() == 0:
    print("recency_cancel done")
    
#tot_cancel_pols can change to 0 (stands for no policy cancelled).
df['tot_cancel_pols'].fillna(0, inplace=True)
if df["tot_cancel_pols"].isnull().sum() == 0:
    print("tot_cancel_pols done")

In [ ]:
#other lapse features: lapse_ape_ltc_1280bf, lapse_ape_grp_6fc3e6, lapse_ape_grp_de05ae, lapse_ape_inv_dcd836, lapse_ape_grp_945b5a, 
#                      lapse_ape_grp_6a5788, lapse_ape_ltc_43b9d5, lapse_ape_grp_9cdedf, lapse_ape_lh_d0adeb, lapse_ape_grp_1581d7, 
#                      lapse_ape_grp_22decf, lapse_ape_lh_507c37, lapse_ape_lh_839f8a, lapse_ape_inv_e9f316, lapse_ape_grp_caa6ff, 
#                      lapse_ape_grp_fd3bfb, lapse_ape_lh_e22a6a, lapse_ape_grp_70e1dd, lapse_ape_grp_e04c3a, lapse_ape_grp_fe5fb8, 
#                      lapse_ape_grp_94baec, lapse_ape_grp_e91421, lapse_ape_lh_f852af, lapse_ape_lh_947b15, lapse_ape_32c74c, 
#                      n_months_since_lapse_ltc_1280bf, n_months_since_lapse_grp_6fc3e6, n_months_since_lapse_grp_de05ae, 
#                      n_months_since_lapse_inv_dcd836, n_months_since_lapse_grp_945b5a, n_months_since_lapse_grp_6a5788, 
#                      n_months_since_lapse_ltc_43b9d5, n_months_since_lapse_grp_9cdedf, n_months_since_lapse_lh_d0adeb, 
#                      n_months_since_lapse_grp_1581d7, n_months_since_lapse_grp_22decf, n_months_since_lapse_lh_507c37, 
#                      n_months_since_lapse_lh_839f8a, n_months_since_lapse_inv_e9f316, n_months_since_lapse_grp_caa6ff, 
#                      n_months_since_lapse_grp_fd3bfb, n_months_since_lapse_lh_e22a6a, n_months_since_lapse_grp_70e1dd, 
#                      n_months_since_lapse_grp_e04c3a, n_months_since_lapse_grp_fe5fb8, n_months_since_lapse_grp_94baec, 
#                      n_months_since_lapse_grp_e91421, n_months_since_lapse_lh_f852af, n_months_since_lapse_lh_947b15, 
#                      n_months_since_lapse_32c74c

#drop the lapse_ape columns that have less than 10 different values (nonzero, nonnull)
#drop lapse_ape_ltc_1280bf (0), lapse_ape_grp_de05ae (1), lapse_ape_inv_dcd836 (0), lapse_ape_lh_d0adeb (0),
#     lapse_ape_inv_e9f316 (5), lapse_ape_32c74c (0)
#for the others, convert to cat var (category for 0s; replace None with "No lapse")

#drop the n_months_since_lapse columns that have less than 10 different values (non-ve, non-9999, nonnull)
#drop n_months_since_lapse_ltc_1280bf (0), n_months_since_lapse_grp_de05ae (2), n_months_since_lapse_inv_dcd836 (0),
#     n_months_since_lapse_lh_d0adeb (0), n_months_since_lapse_inv_e9f316 (7), n_months_since_lapse_32c74c (0)
#for the others, convert to cat var (replace -ve, 9999 with "No data"; replace None with "No lapse")

#dropping the lapse_ape and n_months_since_lapse from above
cols = ["lapse_ape_ltc_1280bf","lapse_ape_grp_de05ae","lapse_ape_inv_dcd836","lapse_ape_lh_d0adeb","lapse_ape_inv_e9f316",
       "lapse_ape_32c74c","n_months_since_lapse_ltc_1280bf","n_months_since_lapse_grp_de05ae","n_months_since_lapse_inv_dcd836",
       "n_months_since_lapse_lh_d0adeb","n_months_since_lapse_inv_e9f316","n_months_since_lapse_32c74c"]
a = df.shape[1]
df.drop(cols, axis=1, inplace=True)
if df.shape[1] + 12 == a:
    print("Dropped 12 columns")
print(df.shape)

#lapse_ape_grp_6fc3e6
bins = [0, 0.01, 100, 200, float('inf')]
labels = ['Zero', '>0-100', '101-200', '>200']
df['lapse_ape_grp_6fc3e6'].fillna(-1, inplace=True)
df['lapse_ape_grp_6fc3e6'] = pd.cut(df['lapse_ape_grp_6fc3e6'], bins=bins, labels=labels, right=False)
df['lapse_ape_grp_6fc3e6'] = df['lapse_ape_grp_6fc3e6'].cat.add_categories("No lapse").fillna("No lapse")
if df["lapse_ape_grp_6fc3e6"].isnull().sum() == 0:
    print("lapse_ape_grp_6fc3e6 done")

#lapse_ape_grp_945b5a
bins = [0, 0.01, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, float('inf')]
labels = ['Zero', '>0-100', '101-200', '201-300', '301-400', '401-500', '501-600', '601-700', '701-800', '801-900', '901-1000', '>1000']
df['lapse_ape_grp_945b5a'].fillna(-1, inplace=True)
df['lapse_ape_grp_945b5a'] = pd.cut(df['lapse_ape_grp_945b5a'], bins=bins, labels=labels, right=False)
df['lapse_ape_grp_945b5a'] = df['lapse_ape_grp_945b5a'].cat.add_categories("No lapse").fillna("No lapse")
if df["lapse_ape_grp_945b5a"].isnull().sum() == 0:
    print("lapse_ape_grp_945b5a done")

#lapse_ape_grp_6a5788 
bins = [0, 0.01, 200, 400, 600, float('inf')]
labels = ['Zero', '>0-200', '201-400', '401-600', '>600']
df['lapse_ape_grp_6a5788'].fillna(-1, inplace=True)
df['lapse_ape_grp_6a5788'] = pd.cut(df['lapse_ape_grp_6a5788'], bins=bins, labels=labels, right=False)
df['lapse_ape_grp_6a5788'] = df['lapse_ape_grp_6a5788'].cat.add_categories("No lapse").fillna("No lapse")
if df["lapse_ape_grp_6a5788"].isnull().sum() == 0:
    print("lapse_ape_grp_6a5788 done")
    
#lapse_ape_ltc_43b9d5 
bins = [0, 0.01, 200, 400, 600, 800, 1000, float('inf')]
labels = ['Zero', '>0-200', '201-400', '401-600', '601-800', '801-1000', '>1000']
df['lapse_ape_ltc_43b9d5'].fillna(-1, inplace=True)
df['lapse_ape_ltc_43b9d5'] = pd.cut(df['lapse_ape_ltc_43b9d5'], bins=bins, labels=labels, right=False)
df['lapse_ape_ltc_43b9d5'] = df['lapse_ape_ltc_43b9d5'].cat.add_categories("No lapse").fillna("No lapse")
if df["lapse_ape_ltc_43b9d5"].isnull().sum() == 0:
    print("lapse_ape_ltc_43b9d5 done")
    
#lapse_ape_grp_9cdedf 
bins = [0, 0.01, 100, 200, 300, 400, 500, float('inf')]
labels = ['Zero', '>0-100', '101-200', '201-300', '301-400', '401-500', '>500']
df['lapse_ape_grp_9cdedf'].fillna(-1, inplace=True)
df['lapse_ape_grp_9cdedf'] = pd.cut(df['lapse_ape_grp_9cdedf'], bins=bins, labels=labels, right=False)
df['lapse_ape_grp_9cdedf'] = df['lapse_ape_grp_9cdedf'].cat.add_categories("No lapse").fillna("No lapse")
if df["lapse_ape_grp_9cdedf"].isnull().sum() == 0:
    print("lapse_ape_grp_9cdedf done")
    
#lapse_ape_grp_1581d7
bins = [0, 0.01, 50, 100, 150, 200, 250, float('inf')]
labels = ['Zero', '>0-50', '51-100', '101-150', '151-200', '200-250', '>250']
df['lapse_ape_grp_1581d7'].fillna(-1, inplace=True)
df['lapse_ape_grp_1581d7'] = pd.cut(df['lapse_ape_grp_1581d7'], bins=bins, labels=labels, right=False)
df['lapse_ape_grp_1581d7'] = df['lapse_ape_grp_1581d7'].cat.add_categories("No lapse").fillna("No lapse")
if df["lapse_ape_grp_1581d7"].isnull().sum() == 0:
    print("lapse_ape_grp_1581d7 done")
    
#lapse_ape_grp_22decf
bins = [0, 0.01, 500, 1000, 1500, 2000, float('inf')]
labels = ['Zero', '>0-500', '501-1000', '1001-1500', '1501-2000','>2000']
df['lapse_ape_grp_22decf'].fillna(-1, inplace=True)
df['lapse_ape_grp_22decf'] = pd.cut(df['lapse_ape_grp_22decf'], bins=bins, labels=labels, right=False)
df['lapse_ape_grp_22decf'] = df['lapse_ape_grp_22decf'].cat.add_categories("No lapse").fillna("No lapse")
if df["lapse_ape_grp_22decf"].isnull().sum() == 0:
    print("lapse_ape_grp_22decf done")
    
#lapse_ape_lh_507c37
bins = [0, 0.01, 500, 1000, 1500, 2000, 3000, 4000, 5000, float('inf')]
labels = ['Zero', '>0-500', '501-1000', '1001-1500', '1501-2000','2001-3000','3001-4000','4001-5000','>5000']
df['lapse_ape_lh_507c37'].fillna(-1, inplace=True)
df['lapse_ape_lh_507c37'] = pd.cut(df['lapse_ape_lh_507c37'], bins=bins, labels=labels, right=False)
df['lapse_ape_lh_507c37'] = df['lapse_ape_lh_507c37'].cat.add_categories("No lapse").fillna("No lapse")
if df["lapse_ape_lh_507c37"].isnull().sum() == 0:
    print("lapse_ape_lh_507c37 done")

#lapse_ape_lh_839f8a
bins = [0, 0.01, 1000, 1500, 2000, 2500, 3000, 4000, 5000, float('inf')]
labels = ['Zero', '>0-1000', '1001-1500', '1501-2000', '2001-2500', '2501-3000', '3001-4000', '4001-5000', '>5000']
df['lapse_ape_lh_839f8a'].fillna(-1, inplace=True)
df['lapse_ape_lh_839f8a'] = pd.cut(df['lapse_ape_lh_839f8a'], bins=bins, labels=labels, right=False)
df['lapse_ape_lh_839f8a'] = df['lapse_ape_lh_839f8a'].cat.add_categories("No lapse").fillna("No lapse")
if df["lapse_ape_lh_839f8a"].isnull().sum() == 0:
    print("lapse_ape_lh_839f8a done")
    
#lapse_ape_grp_caa6ff
bins = [0, 0.01, 100, 200, 300, 400, 500, 600, 700, float('inf')]
labels = ['Zero', '>0-100', '101-200', '201-300', '301-400', '401-500', '501-600', '601-700', '>700']
df['lapse_ape_grp_caa6ff'].fillna(-1, inplace=True)
df['lapse_ape_grp_caa6ff'] = pd.cut(df['lapse_ape_grp_caa6ff'], bins=bins, labels=labels, right=False)
df['lapse_ape_grp_caa6ff'] = df['lapse_ape_grp_caa6ff'].cat.add_categories("No lapse").fillna("No lapse")
if df["lapse_ape_grp_caa6ff"].isnull().sum() == 0:
    print("lapse_ape_grp_caa6ff done")
    
#lapse_ape_grp_fd3bfb
bins = [0, 0.01, 200, 400, 600, float('inf')]
labels = ['Zero', '>0-200', '201-400', '401-600', '>600']
df['lapse_ape_grp_fd3bfb'].fillna(-1, inplace=True)
df['lapse_ape_grp_fd3bfb'] = pd.cut(df['lapse_ape_grp_fd3bfb'], bins=bins, labels=labels, right=False)
df['lapse_ape_grp_fd3bfb'] = df['lapse_ape_grp_fd3bfb'].cat.add_categories("No lapse").fillna("No lapse")
if df["lapse_ape_grp_fd3bfb"].isnull().sum() == 0:
    print("lapse_ape_grp_fd3bfb done")
    
#lapse_ape_lh_e22a6a
bins = [0, 0.01, 400, 800, 1200, 1600, 2000, 3000, 4000, float('inf')]
labels = ['Zero', '>0-400', '401-800', '801-1200', '1201-1600', '1601-2000', '2001-3000', '3001-4000', '>4000']
df['lapse_ape_lh_e22a6a'].fillna(-1, inplace=True)
df['lapse_ape_lh_e22a6a'] = pd.cut(df['lapse_ape_lh_e22a6a'], bins=bins, labels=labels, right=False)
df['lapse_ape_lh_e22a6a'] = df['lapse_ape_lh_e22a6a'].cat.add_categories("No lapse").fillna("No lapse")
if df["lapse_ape_lh_e22a6a"].isnull().sum() == 0:
    print("lapse_ape_lh_e22a6a done")
    
#lapse_ape_grp_70e1dd
bins = [0, 0.01, 100, 200, 300, 400, 500, 600, float('inf')]
labels = ['Zero', '>0-100', '101-200', '201-300', '301-400', '401-500', '501-600', '>600']
df['lapse_ape_grp_70e1dd'].fillna(-1, inplace=True)
df['lapse_ape_grp_70e1dd'] = pd.cut(df['lapse_ape_grp_70e1dd'], bins=bins, labels=labels, right=False)
df['lapse_ape_grp_70e1dd'] = df['lapse_ape_grp_70e1dd'].cat.add_categories("No lapse").fillna("No lapse")
if df["lapse_ape_grp_70e1dd"].isnull().sum() == 0:
    print("lapse_ape_grp_70e1dd done")
    
#lapse_ape_grp_e04c3a
bins = [0, 0.01, 500, 1000, 1500, 2000, float('inf')]
labels = ['Zero', '>0-500', '501-1000', '1001-1500', '1501-2000', '>2000']
df['lapse_ape_grp_e04c3a'].fillna(-1, inplace=True)
df['lapse_ape_grp_e04c3a'] = pd.cut(df['lapse_ape_grp_e04c3a'], bins=bins, labels=labels, right=False)
df['lapse_ape_grp_e04c3a'] = df['lapse_ape_grp_e04c3a'].cat.add_categories("No lapse").fillna("No lapse")
if df["lapse_ape_grp_e04c3a"].isnull().sum() == 0:
    print("lapse_ape_grp_e04c3a done")
    
#lapse_ape_grp_fe5fb8
bins = [0, 0.01, 200, 400, 600, 800, 1000, 1200, 1400, 1600, float('inf')]
labels = ['Zero', '>0-200', '201-400', '401-600', '601-800', '801-1000', '1001-1200', '1201-1400', '1401-1600', '>1600']
df['lapse_ape_grp_fe5fb8'].fillna(-1, inplace=True)
df['lapse_ape_grp_fe5fb8'] = pd.cut(df['lapse_ape_grp_fe5fb8'], bins=bins, labels=labels, right=False)
df['lapse_ape_grp_fe5fb8'] = df['lapse_ape_grp_fe5fb8'].cat.add_categories("No lapse").fillna("No lapse")
if df["lapse_ape_grp_fe5fb8"].isnull().sum() == 0:
    print("lapse_ape_grp_fe5fb8 done")
    
#lapse_ape_grp_94baec
bins = [0, 0.01, 100, 200, 400, 600, 800, 1000, float('inf')]
labels = ['Zero', '>0-100', '101-200', '201-400', '401-600', '601-800', '801-1000', '>1000']
df['lapse_ape_grp_94baec'].fillna(-1, inplace=True)
df['lapse_ape_grp_94baec'] = pd.cut(df['lapse_ape_grp_94baec'], bins=bins, labels=labels, right=False)
df['lapse_ape_grp_94baec'] = df['lapse_ape_grp_94baec'].cat.add_categories("No lapse").fillna("No lapse")
if df["lapse_ape_grp_94baec"].isnull().sum() == 0:
    print("lapse_ape_grp_94baec done")
    
#lapse_ape_grp_e91421
bins = [0, 0.01, 200, 400, 600, 800, 1000, float('inf')]
labels = ['Zero', '>0-200', '201-400', '401-600', '601-800', '801-1000', '>1000']
df['lapse_ape_grp_e91421'].fillna(-1, inplace=True)
df['lapse_ape_grp_e91421'] = pd.cut(df['lapse_ape_grp_e91421'], bins=bins, labels=labels, right=False)
df['lapse_ape_grp_e91421'] = df['lapse_ape_grp_e91421'].cat.add_categories("No lapse").fillna("No lapse")
if df["lapse_ape_grp_e91421"].isnull().sum() == 0:
    print("lapse_ape_grp_e91421 done")
    
#lapse_ape_lh_f852af
bins = [0, 0.01, 500, 1000, 1500, 2000, 2500, 3000, 4000, 6000, 8000, float('inf')]
labels = ['Zero', '>0-500', '501-1000', '1001-1500', '1501-2000', '2001-2500', '2501-3000', '3001-4000', '4001-6000', '6001-8000', '>8000']
df['lapse_ape_lh_f852af'].fillna(-1, inplace=True)
df['lapse_ape_lh_f852af'] = pd.cut(df['lapse_ape_lh_f852af'], bins=bins, labels=labels, right=False)
df['lapse_ape_lh_f852af'] = df['lapse_ape_lh_f852af'].cat.add_categories("No lapse").fillna("No lapse")
if df["lapse_ape_lh_f852af"].isnull().sum() == 0:
    print("lapse_ape_lh_f852af done")
    
#lapse_ape_lh_947b15
bins = [0, 0.01, 1000, 2000, 3000, 4000, 5000, 6000, float('inf')]
labels = ['Zero', '>0-1000', '1001-2000', '2001-3000', '3001-4000', '4001-5000', '5001-6000', '>6000']
df['lapse_ape_lh_947b15'].fillna(-1, inplace=True)
df['lapse_ape_lh_947b15'] = pd.cut(df['lapse_ape_lh_947b15'], bins=bins, labels=labels, right=False)
df['lapse_ape_lh_947b15'] = df['lapse_ape_lh_947b15'].cat.add_categories("No lapse").fillna("No lapse")
if df["lapse_ape_lh_947b15"].isnull().sum() == 0:
    print("lapse_ape_lh_947b15 done")
    
#converting n_months_since_lapse columns to cat var
cols=['n_months_since_lapse_grp_6fc3e6', 'n_months_since_lapse_grp_945b5a', 'n_months_since_lapse_grp_6a5788', 
'n_months_since_lapse_ltc_43b9d5', 'n_months_since_lapse_grp_9cdedf',
'n_months_since_lapse_grp_1581d7', 'n_months_since_lapse_grp_22decf', 'n_months_since_lapse_lh_507c37', 
'n_months_since_lapse_lh_839f8a', 'n_months_since_lapse_grp_caa6ff', 
'n_months_since_lapse_grp_fd3bfb', 'n_months_since_lapse_lh_e22a6a', 'n_months_since_lapse_grp_70e1dd', 
'n_months_since_lapse_grp_e04c3a', 'n_months_since_lapse_grp_fe5fb8', 'n_months_since_lapse_grp_94baec', 
'n_months_since_lapse_grp_e91421', 'n_months_since_lapse_lh_f852af', 'n_months_since_lapse_lh_947b15']

bins = [0, 20, 40, 60, 80, 100, float('inf')]
labels = ['0-20', '21-40', '41-60', '61-80', '81-100', '>100']

for col in cols:
    df[col] = df[col].astype('float64').replace(9999,-1)
    null_indices = df[col].index[df[col].isnull()].tolist()
    df[col] = pd.cut(df[col], bins=bins, labels=labels, right=False)
    df[col] = df[col].cat.add_categories("No lapse")
    df[col].loc[null_indices] = "No lapse"
    df[col] = df[col].cat.add_categories("No data").fillna("No data")
    if df[col].isnull().sum() == 0:
        print(f"{col} done")

In [ ]:
#checking the distribution of columns, for determining the bins when converting to categorical variable

print(df['giclaim_amt'].value_counts())

plt.hist(df['recency_giclaim'], bins=50, color='skyblue', edgecolor='black')
plt.title('Histogram of "your_column"')
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.show()

In [ ]:
#flg_affconnect_show_interest_ever can change to False (1 change to True)
df['flg_affconnect_show_interest_ever'].fillna(False, inplace=True)
df['flg_affconnect_show_interest_ever'] = df['flg_affconnect_show_interest_ever'].astype(bool)
if df["flg_affconnect_show_interest_ever"].isnull().sum() == 0:
    print("flg_affconnect_show_interest_ever done")

#flg_affconnect_ready_to_buy_ever can be dropped (very similar to n_months_since_visit_affcon)
a = df.shape[1]
df.drop("flg_affconnect_ready_to_buy_ever", axis=1, inplace=True)
if df.shape[1] + 1 == a:
    print("Dropped flg_affconnect_ready_to_buy_ever")
print(df.shape)

#flg_affconnect_lapse_ever can be dropped (duplicate of have_lapse and flg_affconnect_ready_to_buy_ever)
a = df.shape[1]
df.drop("flg_affconnect_lapse_ever", axis=1, inplace=True)
if df.shape[1] + 1 == a:
    print("Dropped flg_affconnect_lapse_ever")
print(df.shape)

#affcon_visit_days can be changed to 0
    #Here we can hypothesise that 16176 people did not buy through affcon
df['affcon_visit_days'].fillna(0, inplace=True)
if df["affcon_visit_days"].isnull().sum() == 0:
    print("affcon_visit_days done")
    
#n_months_since_visit_affcon can be converted to cat var, replace nan with "no affcon".
df['n_months_since_visit_affcon'] = df['n_months_since_visit_affcon'].astype('category')
df['n_months_since_visit_affcon'] = df['n_months_since_visit_affcon'].cat.add_categories("No affcon").fillna("No affcon")
if df["n_months_since_visit_affcon"].isnull().sum() == 0:
    print("n_months_since_visit_affcon done")

#clmcon_visit_days can be changed to 0
    #Here we can hypothesise that 16279 people did not use clmcon
df['clmcon_visit_days'].fillna(0, inplace=True)
if df["clmcon_visit_days"].isnull().sum() == 0:
    print("clmcon_visit_days done")
    
#recency_clmcon, recency_clmcon_regis can be converted to cat var, replace nan with "no clmcon"
    #recency_clmcon_regis can use bins of range 20
df['recency_clmcon'] = df['recency_clmcon'].astype('category')
df['recency_clmcon'] = df['recency_clmcon'].cat.add_categories("No clmcon").fillna("No clmcon")
if df["recency_clmcon"].isnull().sum() == 0:
    print("recency_clmcon done")

bins = [0, 10, 20, 30, 40, 60, 80, float('inf')]
labels = ['0-10', '11-20', '21-30', '31-40', '41-60', '61-80', '>80']
df['recency_clmcon_regis'] = pd.cut(df['recency_clmcon_regis'], bins=bins, labels=labels, right=False)
df['recency_clmcon_regis'] = df['recency_clmcon_regis'].cat.add_categories("No clmcon").fillna("No clmcon")
if df["recency_clmcon_regis"].isnull().sum() == 0:
    print("recency_clmcon_regis done")
    
#hlthclaim_amt can be converted to cat var, replace nan with "no hlthclaim"
    #Low: $0 - $500; Medium Low: $501 - $1,000; Medium: $1,001 - $5,000; High: $5,001 - $10,000; Very High: $10,001 and above
    #Here we can hypothesise that 15552 people did not make healthclaims
bins = [0, 500, 1000, 5000, 10000, float('inf')]
labels = ['Low: $0 - $500', 'Medium Low: $501 - $1,000', 'Medium: $1,001 - $5,000', 'High: $5,001 - $10,000', 'Very High: >$10,000']
df['hlthclaim_amt'] = df['hlthclaim_amt'].fillna(-1)
df['hlthclaim_amt'] = pd.cut(df['hlthclaim_amt'], bins=bins, labels=labels, right=False)
df['hlthclaim_amt'] = df['hlthclaim_amt'].cat.add_categories("No healthclaim").fillna("No healthclaim")
if df["hlthclaim_amt"].isnull().sum() == 0:
    print("hlthclaim_amt done")
    
#recency_hlthclaim can be converted to cat var, replace nan with "no hlthclaim"
    #can use bins of range 20
bins = [0, 10, 20, 30, 40, 60, 80, 100, float('inf')]
labels = ['0-10', '11-20', '21-30', '31-40', '41-60', '61-80', '81-100', '>100']
df['recency_hlthclaim'] = pd.cut(df['recency_hlthclaim'], bins=bins, labels=labels, right=False)
df['recency_hlthclaim'] = df['recency_hlthclaim'].cat.add_categories("No healthclaim").fillna("No healthclaim")
if df["recency_hlthclaim"].isnull().sum() == 0:
    print("recency_hlthclaim done")
    
#hlthclaim_cnt_success can be converted to cat var, replace nan with "no success"
    #can use bins of ranges 0-20; 21-40; 41-100; 101-180; >180
bins = [0, 20, 40, 100, 180, float('inf')]
labels = ['0-20', '21-40', '41-100', '101-180', '>180']
df['hlthclaim_cnt_success'] = pd.cut(df['hlthclaim_cnt_success'], bins=bins, labels=labels, right=False)
df['hlthclaim_cnt_success'] = df['hlthclaim_cnt_success'].cat.add_categories("No success").fillna("No success")
if df["hlthclaim_cnt_success"].isnull().sum() == 0:
    print("hlthclaim_cnt_success done")
    
#recency_hlthclaim_success can be converted to cat var, replace nan with "no success"
    #can use bins of range 20
bins = [0, 10, 20, 30, 40, 60, 80, 100, float('inf')]
labels = ['0-10', '11-20', '21-30', '31-40', '41-60', '61-80', '81-100', '>100']
df['recency_hlthclaim_success'] = pd.cut(df['recency_hlthclaim_success'], bins=bins, labels=labels, right=False)
df['recency_hlthclaim_success'] = df['recency_hlthclaim_success'].cat.add_categories("No success").fillna("No success")
if df["recency_hlthclaim_success"].isnull().sum() == 0:
    print("recency_hlthclaim_success done")
    
#hlthclaim_cnt_unsuccess can be converted to cat var, replace nan with "no unsuccess"
bins = [0, 1, 2, 4, 6, 8, 10, 15, float('inf')]
labels = ['1', '2', '3-4', '5-6', '7-8', '9-10', '11-15', '>15']
df['hlthclaim_cnt_unsuccess'] = pd.cut(df['hlthclaim_cnt_unsuccess'], bins=bins, labels=labels, right=False)
df['hlthclaim_cnt_unsuccess'] = df['hlthclaim_cnt_unsuccess'].cat.add_categories("No unsuccess").fillna("No unsuccess")
if df["hlthclaim_cnt_unsuccess"].isnull().sum() == 0:
    print("hlthclaim_cnt_unsuccess done")
    
#recency_hlthclaim_unsuccess can be converted to cat var, replace nan with "no unsuccess"
    #can use bins of range 20
bins = [0, 20, 40, 60, 80, 100, 120, float('inf')]
labels = ['0-20', '21-40', '41-60', '61-80', '81-100', '101-120', '>120']
df['recency_hlthclaim_unsuccess'] = pd.cut(df['recency_hlthclaim_unsuccess'], bins=bins, labels=labels, right=False)
df['recency_hlthclaim_unsuccess'] = df['recency_hlthclaim_unsuccess'].cat.add_categories("No unsuccess").fillna("No unsuccess")
if df["recency_hlthclaim_unsuccess"].isnull().sum() == 0:
    print("recency_hlthclaim_unsuccess done")

#flg_hlthclaim_839f8a_ever can change to False (1 change to True)
df['flg_hlthclaim_839f8a_ever'].fillna(False, inplace=True)
df['flg_hlthclaim_839f8a_ever'] = df['flg_hlthclaim_839f8a_ever'].astype(bool)
if df["flg_hlthclaim_839f8a_ever"].isnull().sum() == 0:
    print("flg_hlthclaim_839f8a_ever done")

#recency_hlthclaim_839f8a can be converted to cat var, replace nan with "no claim"
bins = [0, 2, 5, 10, 20, 40, 60, 80, 100, 120, float('inf')]
labels = ['0-2', '3-5', '6-10', '11-20', '21-40', '41-60', '61-80', '81-100', '101-120', '>120']
df['recency_hlthclaim_839f8a'] = pd.cut(df['recency_hlthclaim_839f8a'], bins=bins, labels=labels, right=False)
df['recency_hlthclaim_839f8a'] = df['recency_hlthclaim_839f8a'].cat.add_categories("No claim").fillna("No claim")
if df["recency_hlthclaim_839f8a"].isnull().sum() == 0:
    print("recency_hlthclaim_839f8a done")
    
#flg_hlthclaim_14cb37_ever can change to False (1 change to True)
df['flg_hlthclaim_14cb37_ever'].fillna(False, inplace=True)
df['flg_hlthclaim_14cb37_ever'] = df['flg_hlthclaim_14cb37_ever'].astype(bool)
if df["flg_hlthclaim_14cb37_ever"].isnull().sum() == 0:
    print("flg_hlthclaim_14cb37_ever done")

#recency_hlthclaim_14cb37 can be converted to cat var, replace nan with "no claim"
bins = [0, 2, 5, 10, 20, 40, 60, 80, 100, 120, float('inf')]
labels = ['0-2', '3-5', '6-10', '11-20', '21-40', '41-60', '61-80', '81-100', '101-120', '>120']
df['recency_hlthclaim_14cb37'] = pd.cut(df['recency_hlthclaim_14cb37'], bins=bins, labels=labels, right=False)
df['recency_hlthclaim_14cb37'] = df['recency_hlthclaim_14cb37'].cat.add_categories("No claim").fillna("No claim")
if df["recency_hlthclaim_14cb37"].isnull().sum() == 0:
    print("recency_hlthclaim_14cb37 done")
    
#giclaim_amt can be converted to cat var, replace nan with "no giclaim"
    #Low: $0 - $500; Medium Low: $501 - $1,000; Medium: $1,001 - $5,000; High: $5,001 - $10,000; Very High: $10,001 and above
    #Here we can hypothesise that 16545 people did not make giclaims
bins = [0, 500, 1000, 5000, 10000, float('inf')]
labels = ['Low: $0 - $500', 'Medium Low: $501 - $1,000', 'Medium: $1,001 - $5,000', 'High: $5,001 - $10,000', 'Very High: >$10,000']
df['giclaim_amt'] = df['giclaim_amt'].fillna(-1)
df['giclaim_amt'] = pd.cut(df['giclaim_amt'], bins=bins, labels=labels, right=False)
df['giclaim_amt'] = df['giclaim_amt'].cat.add_categories("No giclaim").fillna("No giclaim")
if df["giclaim_amt"].isnull().sum() == 0:
    print("giclaim_amt done")
    
#recency_giclaim can be converted to cat var, replace nan with "no giclaim"
    #can use bins of range 10
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, float('inf')]
labels = ['0-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100', '>100']
df['recency_giclaim'] = pd.cut(df['recency_giclaim'], bins=bins, labels=labels, right=False)
df['recency_giclaim'] = df['recency_giclaim'].cat.add_categories("No giclaim").fillna("No giclaim")
if df["recency_giclaim"].isnull().sum() == 0:
    print("recency_giclaim done")
    
#f_purchase_lh can change to False (1 change to True)
df['f_purchase_lh'].fillna(False, inplace=True)
df['f_purchase_lh'] = df['f_purchase_lh'].astype(bool)
if df["f_purchase_lh"].isnull().sum() == 0:
    print("f_purchase_lh done")

In [ ]:
if df.isnull().sum().sum()==0:
    print("All null values cleared!")

In [ ]:
# Function to convert object columns to categorical
def convert_to_categorical(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].astype('category')
    return df

# Convert pandas DataFrame to categorical
df = convert_to_categorical(df)

# Initialize H2O
h2o.init()

# Convert pandas DataFrame to H2OFrame
h2o_df = h2o.H2OFrame(df)

# Split the dataset into a train and valid set:
train, valid = h2o_df.split_frame(ratios=[.8], seed=1234)

# Training the Model
RF_Model = H2ORandomForestEstimator(ntrees=10,
                                    max_depth=5,
                                    min_rows=10,
                                    calibrate_model=True,
                                    calibration_frame=valid,
                                    binomial_double_trees=True)

# Train the Random Forest Model
RF_Model.train(x=list(df.columns).remove('f_purchase_lh'),
               y='f_purchase_lh',
               training_frame=train,
               validation_frame=valid)

model_path = h2o.save_model(model=RF_Model, path="./model", force=True)
print(model_path)

# Ensure the input_data is an H2O Frame
if not isinstance(df, h2o.H2OFrame):
    input_data = h2o.H2OFrame(df)

# Eval performance:
perf = RF_Model.model_performance()

print("Performance: \n", perf)
# Generate predictions on a validation set (if necessary):
pred = RF_Model.predict(valid)

print("Prediction: \n", pred)

### Please have the filename renamed and ensure that it can be run with the requirements above being met. All the best!